# PySpark DataFrames & MLlib introduction

## Part I - DataFrames

*Great tutorial for installing pyspark on Jupyter.*

https://bigdata-madesimple.com/guide-to-install-spark-and-use-pyspark-from-jupyter-in-windows/

*Test 1*

In [2]:
import findspark
findspark.init()

*Test 2*

In [3]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello ")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



### PySpark x Python syntax differences

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession #PySpark has to firstly import session 
spark = SparkSession.builder.appName("PySpart").getOrCreate()

In [5]:
spark

In [6]:
data = [["tom",14],["janos",16],["janet",15]]

In [9]:
df = spark.createDataFrame(data,["Name","Age"]) #creating table in Spark

In [11]:
df.show() 

+-----+---+
| Name|Age|
+-----+---+
|  tom| 14|
|janos| 16|
|janet| 15|
+-----+---+



In [13]:
df.toPandas() #better for vizualisation!

,Name,Age
0,tom,14
1,janos,16
2,janet,15


In [15]:
df.columns #same as in pandas

['Name', 'Age']

In [17]:
df.count() #pandas has len 

3

In [ ]:
path = "students.csv"
# reading with spark df = spark.read.csv(path, header True)

In [23]:
df.groupBy("Name").agg({"Age": "mean"}).show()

+-----+--------+
| Name|avg(Age)|
+-----+--------+
|janet|    15.0|
|janos|    16.0|
|  tom|    14.0|
+-----+--------+



In [24]:
from pyspark.sql import functions as F #for using multiple agg functions for one ..

*When we chnage the table content - we are creating a brand new copy, with new id. Tables are immutable.*

## Part II Read, Write, Validate & Explore

- PysPARK is dividing DataFrame into more parts, so we can use only parts we require.
- Spark uses UDF's functions in some cases, we cannot use classic Python function for iterating through our data.

In [29]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("OptionalName").getOrCreate() #get or create not zero :)
#its just basic imports + creating a session so I can have table

In [27]:
spark

*Our Session is now in memory, our Spark UI opens new table, where we can see our Jobs.*

In [31]:
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

In [32]:
cores

1

Just chechikng how many cores Spark is using.

### Loading data

<div class="alert alert-block alert-warning"><b>CSV</b></div>

In [33]:
path = "Datasets/"

students = spark.read.csv(path+"students.csv", inferSchema=True, header=True)
#inferSchema I use instead of writing what type of file my file is ..not workikng all the time

In [34]:
students.columns #classic introspection

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

In [35]:
students.limit(4).toPandas() #instead of head I can use limit function and I have to use - toPandas for better 
#looking table

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44


<div class="alert alert-block alert-warning"><b>Parquet data</b></div>

In [38]:
parquet = spark.read.parquet(path+"users1.parquet") #reading only one parquet file

In [39]:
parquet.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 08:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 18:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 02:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 01:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


<div class="alert alert-block alert-warning"><b>Multiple parquet</b></div>

In [46]:
partitioned = spark.read.parquet(path+"users2.parquet", path+"users2.parquet") #users and asterix - meaning it will pick up everything

In [48]:
partitioned.limit(4).toPandas()

,registration_dttm,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,2016-02-03 08:55:29,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,1E+02
1,2016-02-03 18:04:03,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,
2,2016-02-03 02:09:31,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,
3,2016-02-03 01:36:21,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,


<div class="alert alert-block alert-warning"><b>Summary functions</b></div>

In [50]:
students.printSchema()

root
 |-- gender: string (nullable = true)
 |-- race/ethnicity: string (nullable = true)
 |-- parental level of education: string (nullable = true)
 |-- lunch: string (nullable = true)
 |-- test preparation course: string (nullable = true)
 |-- math score: integer (nullable = true)
 |-- reading score: integer (nullable = true)
 |-- writing score: integer (nullable = true)



*Very similar to info() function in Pandas.*

In [53]:
students.describe() #similar to printSchema
#nullable = true - ABLE TO HOLD NULL VALUES

DataFrame[summary: string, gender: string, race/ethnicity: string, parental level of education: string, lunch: string, test preparation course: string, math score: string, reading score: string, writing score: string]

In [52]:
students.schema["math score"].dataType #clear

IntegerType

In [56]:
students.select("math score", "reading score").summary("count", "min", "max").toPandas()

,summary,math score,reading score
0,count,1000,1000
1,min,0,17
2,max,100,100


<div class="alert alert-block alert-warning"><b>Specifying data types</b></div>

In [58]:
from pyspark.sql.types import *

In [71]:
data_schema = [StructField("name", StringType(), True),
               StructField("email", StringType(), True),
               StructField("city", StringType(), True),
               StructField("mac", StringType(), True),
               StructField("timestamp", DateType(), True),
               StructField("creditcard", StringType(), True)]
#creating new schema instead of existing one

In [72]:
final_struc = StructType(fields = data_schema) #creating a schema object

In [73]:
people = spark.read.json(path+"people.json", schema=final_struc) #loading json data with new schema

In [74]:
people.limit(4).toPandas()

,name,email,city,mac,timestamp,creditcard
0,None,None,None,None,None,None
1,Keeley Bosco,katlyn@jenkinsmaggio.net,Lake Gladysberg,08:fd:0b:cd:77:f7,2015-04-25,1228-1221-1221-1431
2,Rubye Jerde,juvenal@johnston.name,None,90:4d:fa:42:63:a2,2015-04-25,1228-1221-1221-1431
3,Miss Darian Breitenberg,None,None,f9:0e:d3:40:cb:e9,2015-04-25,None


In [75]:
people.printSchema()

root
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- city: string (nullable = true)
 |-- mac: string (nullable = true)
 |-- timestamp: date (nullable = true)
 |-- creditcard: string (nullable = true)



<div class="alert alert-block alert-warning"><b>Writing in data</b></div>

In [77]:
data = [["tom",14],["janos",16],["janet",15]]
df = spark.createDataFrame(data,["Name","Age"])

In [78]:
df.toPandas()

,Name,Age
0,tom,14
1,janos,16
2,janet,15


In [79]:
#df.write.mode("overwrite").partitionBy("Season").parquet("partitioned_parqute/")
#just informative syntax for saving files by partitions..

### Search and Filter data

In [80]:
#I will be using import all over again 

In [81]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SearchdF").getOrCreate()

In [82]:
spark

In [84]:
path = "Datasets/"
fifa = spark.read.csv(path+"fifa19.csv", inferSchema=True, header=True)

In [85]:
fifa.limit(4).toPandas() #datasets with 89 columns and 18207 rows

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68,15,21,13,90,85,87,88,94,€138.6M


In [87]:
fifa.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nu

<div class="alert alert-block alert-warning"><b>Select</b></div>

In [89]:
from pyspark.sql.functions import *

In [91]:
fifa.select(["Nationality","Name","Age","Photo"]).show(5) #in pandas without show and without select..

+-----------+-----------------+---+--------------------+
|Nationality|             Name|Age|               Photo|
+-----------+-----------------+---+--------------------+
|  Argentina|         L. Messi| 31|https://cdn.sofif...|
|   Portugal|Cristiano Ronaldo| 33|https://cdn.sofif...|
|     Brazil|        Neymar Jr| 26|https://cdn.sofif...|
|      Spain|           De Gea| 27|https://cdn.sofif...|
|    Belgium|     K. De Bruyne| 27|https://cdn.sofif...|
+-----------+-----------------+---+--------------------+
only showing top 5 rows



In [92]:
fifa.select(["Nationality","Name","Age","Photo"]).show(5, False) #False without truncation

+-----------+-----------------+---+----------------------------------------------+
|Nationality|Name             |Age|Photo                                         |
+-----------+-----------------+---+----------------------------------------------+
|Argentina  |L. Messi         |31 |https://cdn.sofifa.org/players/4/19/158023.png|
|Portugal   |Cristiano Ronaldo|33 |https://cdn.sofifa.org/players/4/19/20801.png |
|Brazil     |Neymar Jr        |26 |https://cdn.sofifa.org/players/4/19/190871.png|
|Spain      |De Gea           |27 |https://cdn.sofifa.org/players/4/19/193080.png|
|Belgium    |K. De Bruyne     |27 |https://cdn.sofifa.org/players/4/19/192985.png|
+-----------+-----------------+---+----------------------------------------------+
only showing top 5 rows



In [95]:
fifa.select(["Name","Age"]).orderBy(fifa["Age"]).show(5) #selection order by another column..

+------------+---+
|        Name|Age|
+------------+---+
|   B. Nygren| 16|
|H. Andersson| 16|
|    A. Doğan| 16|
|  C. Bassett| 16|
|    B. Mumba| 16|
+------------+---+
only showing top 5 rows



In [96]:
fifa.select(["Name","Age"]).orderBy(fifa["Age"].desc()).show(5) #very similar to classic pandas..

+-------------+---+
|         Name|Age|
+-------------+---+
|     O. Pérez| 45|
|K. Pilkington| 44|
|    T. Warner| 44|
|  S. Narazaki| 42|
|    J. Villar| 41|
+-------------+---+
only showing top 5 rows



In [98]:
fifa.select(["Name","Club"]).where(fifa.Club.like("%Barcelona%")).show(5)
#simple filter - just filtering every rows where colum Club containts everything - Barcelona - everything

+---------------+------------+
|           Name|        Club|
+---------------+------------+
|       L. Messi|FC Barcelona|
|      L. Suárez|FC Barcelona|
|  M. ter Stegen|FC Barcelona|
|Sergio Busquets|FC Barcelona|
|       Coutinho|FC Barcelona|
+---------------+------------+
only showing top 5 rows



In [103]:
fifa.select("Photo", fifa.Photo.substr(-4, 3)).show(5,False)
#I am just substring my Photo string - 4 characters from end and show only 3 of them :)

+----------------------------------------------+-----------------------+
|Photo                                         |substring(Photo, -4, 3)|
+----------------------------------------------+-----------------------+
|https://cdn.sofifa.org/players/4/19/158023.png|.pn                    |
|https://cdn.sofifa.org/players/4/19/20801.png |.pn                    |
|https://cdn.sofifa.org/players/4/19/190871.png|.pn                    |
|https://cdn.sofifa.org/players/4/19/193080.png|.pn                    |
|https://cdn.sofifa.org/players/4/19/192985.png|.pn                    |
+----------------------------------------------+-----------------------+
only showing top 5 rows



In [104]:
fifa[fifa.Club.isin("FC Barcelona","Juventus")].limit(3).toPandas() #very basic

,_c0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,FC Barcelona,...,85,62,45,38,27,25,31,33,37,€164M


<div class="alert alert-block alert-warning"><b>Select with Where</b></div>

In [105]:
fifa.select("Name","Club").where(fifa.Name.startswith("L")).where(fifa.Name.endswith("i")).show(5)
#filtering based on Name start with L and end with I

+-------------+---------------+
|         Name|           Club|
+-------------+---------------+
|     L. Messi|   FC Barcelona|
|   L. Bonucci|       Juventus|
| L. Fabiański|West Ham United|
|L. Pellegrini|           Roma|
| L. Pavoletti|       Cagliari|
+-------------+---------------+
only showing top 5 rows



In [106]:
fifa.count() #pandas has classi len

18207

In [107]:
col_list = fifa.columns[0:5]

In [108]:
df2 = fifa.select(col_list)

In [109]:
df2.show(5)

+---+------+-----------------+---+--------------------+
|_c0|    ID|             Name|Age|               Photo|
+---+------+-----------------+---+--------------------+
|  0|158023|         L. Messi| 31|https://cdn.sofif...|
|  1| 20801|Cristiano Ronaldo| 33|https://cdn.sofif...|
|  2|190871|        Neymar Jr| 26|https://cdn.sofif...|
|  3|193080|           De Gea| 27|https://cdn.sofif...|
|  4|192985|     K. De Bruyne| 27|https://cdn.sofif...|
+---+------+-----------------+---+--------------------+
only showing top 5 rows



<div class="alert alert-block alert-warning"><b>Slicing</b></div>

*I can slice in PySpark same as in Pandas.*

In [115]:
df = spark.createDataFrame([([1,2,3],),([4,5],)],["X"]) #just a weird index

In [120]:
df.show()

+---------+
|        X|
+---------+
|[1, 2, 3]|
|   [4, 5]|
+---------+



In [122]:
df.select(slice(df.X,2,2)).show() #this slice is just ignoring 0 nothing else :)

+--------------+
|slice(X, 2, 2)|
+--------------+
|        [2, 3]|
|           [5]|
+--------------+



In [125]:
fifa.filter("Overall>50").select(["Name","Age"]).limit(5).toPandas()#an easy filter

,Name,Age
0,L. Messi,31
1,Cristiano Ronaldo,33
2,Neymar Jr,26
3,De Gea,27
4,K. De Bruyne,27


In [127]:
result = fifa.filter("Overall>50").select(["Nationality","Name","Age","Overall"])\
.orderBy(fifa["Overall"].desc()).collect()

In [130]:
type(result), type(result[0]) #collect is just a lists of rows...

(list, pyspark.sql.types.Row)

In [134]:
print("Worst Player Over 50:", result[-1][1])

Worst Player Over 50: C. Addai


### SQL Options in Spark

In [135]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [139]:
path = "Datasets/"

In [141]:
crime = spark.read.csv(path+"crime.csv", header=True, inferSchema=True)

In [142]:
crime.limit(5).toPandas()

,12 months ending,PFA,Region,Offence,Rolling year total number of offences
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2
4,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561


In [143]:
crime.printSchema()

root
 |-- 12 months ending: string (nullable = true)
 |-- PFA: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Offence: string (nullable = true)
 |-- Rolling year total number of offences: integer (nullable = true)



In [145]:
df = crime.withColumnRenamed("Rolling year total number of offences", "Count") #renaming a column
#park sequel cannot have spaces in column names

In [147]:
df.createOrReplaceTempView("tempview") #creating a tempview - so I can use SQL :)

In [148]:
spark.sql("SELECT * FROM tempview WHERE Count>1000").limit(5).toPandas()
#I am just using simple SQL searching ..

,12 months ending,PFA,Region,Offence,Count
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202
3,31/03/2003,Avon and Somerset,South West,Domestic burglary,14561
4,31/03/2003,Avon and Somerset,South West,Drug offences,2308


In [149]:
from pyspark.ml.feature import SQLTransformer

In [150]:
sqlTrans = SQLTransformer(statement = "SELECT PFA, Region, OFFence FROM __THIS__" )
#__THIS_ IS HERE UNKNOWN 

In [151]:
sqlTrans.transform(df).show(5) #__THIS_ is my df - hence the syntax

+-----------------+----------+--------------------+
|              PFA|    Region|             OFFence|
+-----------------+----------+--------------------+
|Avon and Somerset|South West|All other theft o...|
|Avon and Somerset|South West|       Bicycle theft|
|Avon and Somerset|South West|Criminal damage a...|
|Avon and Somerset|South West|Death or serious ...|
|Avon and Somerset|South West|   Domestic burglary|
+-----------------+----------+--------------------+
only showing top 5 rows



In [152]:
sqlTrans = SQLTransformer(statement = "SELECT Offence, SUM(Count) as TOTAL FROM __THIS__ GROUP BY Offence" )

In [153]:
sqlTrans.transform(df).show(5)

+--------------------+--------+
|             Offence|   TOTAL|
+--------------------+--------+
|Public order offe...|10925676|
|       Bicycle theft| 5297006|
|Residential burglary| 1671469|
|Violence without ...|16590158|
|All other theft o...|30979393|
+--------------------+--------+
only showing top 5 rows



***Bottom line, I can use View or Transformer to access SQL functionality.***

In [168]:
df.select("*", expr("round((count/2444720928)*100,2) AS Percent")).limit(4).toPandas()

,12 months ending,PFA,Region,Offence,Count,Percent
0,31/03/2003,Avon and Somerset,South West,All other theft offences,25959,0.0
1,31/03/2003,Avon and Somerset,South West,Bicycle theft,3090,0.0
2,31/03/2003,Avon and Somerset,South West,Criminal damage and arson,26202,0.0
3,31/03/2003,Avon and Somerset,South West,Death or serious injury caused by illegal driving,2,0.0


The biggest advantage is that user does not need a Python or Scala skills, can just use SQL.

## Clean, Manipulate, Join, Aggregate

### Manipulating Data in DataFrames

In [270]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Manipulate").getOrCreate()
spark

In [271]:
names = spark.createDataFrame([("Abraham","Lincoln")],["first_name","last_name"])

In [272]:
names.show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|   Abraham|  Lincoln|
+----------+---------+



In [273]:
names.rdd.id()

443

In [274]:
from pyspark.sql.functions import *

In [275]:
names = names.select(names.first_name,names.last_name, concat_ws\
                     (" ",names.first_name,names.last_name).alias("full_name"))

In [276]:
names.show() #I just updated new table

+----------+---------+---------------+
|first_name|last_name|      full_name|
+----------+---------+---------------+
|   Abraham|  Lincoln|Abraham Lincoln|
+----------+---------+---------------+



In [277]:
names.rdd.id() #I have new table - different typle...same as always..

449

In [278]:
path = "Datasets/"
videos = spark.read.csv(path+"youtubevideos.csv", inferSchema=True,header=True)

In [279]:
videos.limit(5).toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [280]:
videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [281]:
from pyspark.sql.types import *

In [282]:
df = videos.withColumn("views", videos["views"].cast(IntegerType()))\
.withColumn("likes", videos["likes"].cast(IntegerType()))\
.withColumn("dislikes", videos["dislikes"].cast(IntegerType()))\
.withColumn("trending_date",to_date(videos.trending_date, "yy.dd.mm"))\
.withColumn("publish_time", to_timestamp(videos.publish_time, "yyyy-MM-dd HH:mm:ss"))

*I am using withColumn to overwrite my existing column with new but the same column, BUT with different data types. For regular columns I have cats function (same as in SQL) and to_date or to_timestamp for time columns.*

In [283]:
df.limit(5).toPandas() #Our month and date is switching - which is a little fail ..
#and publish time is NaT which is fail also

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,2017-01-14,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,NaT,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,2017-01-14,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,NaT,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,2017-01-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,NaT,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,2017-01-14,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,NaT,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,2017-01-14,I Dare You: GOING BALD!?,nigahiga,24,NaT,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [284]:
from pyspark.sql.functions import *

In [285]:
videos = videos.withColumn("publish_time_2", regexp_replace(videos.publish_time, "T"," ")) #replacing T

In [286]:
videos = videos.withColumn("publish_time_2", regexp_replace(videos.publish_time_2, "Z","")) #Now Z

In [287]:
videos.limit(5).toPandas()
#so my publish_date_2 is in better format ---> 2017-11-13 17:13:01.000

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_time_2
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,2017-11-13 17:13:01.000
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"""last week tonight trump presidency""|""last wee...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",2017-11-13 07:30:00.000
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"""racist superman""|""rudy""|""mancuso""|""king""|""bac...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,2017-11-12 19:05:24.000
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"""rhett and link""|""gmm""|""good mythical morning""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,2017-11-13 11:00:04.000
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"""ryan""|""higa""|""higatv""|""nigahiga""|""i dare you""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,2017-11-12 18:01:41.000


In [288]:
videos = videos.withColumn("publish_time_3", to_timestamp(videos.publish_time_2, "yyyy-MM-dd HH:mm:ss.SSS"))

In [289]:
videos.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- publish_time_2: string (nullable = true)
 |-- publish_time_3: timestamp (nullable = true)



Now I have publish_time_3 in a correct data type --> timestamp.

In [290]:
videos.select("publish_time", "publish_time_2", "publish_time_3").show(5,False)

+------------------------+-----------------------+-------------------+
|publish_time            |publish_time_2         |publish_time_3     |
+------------------------+-----------------------+-------------------+
|2017-11-13T17:13:01.000Z|2017-11-13 17:13:01.000|2017-11-13 17:13:01|
|2017-11-13T07:30:00.000Z|2017-11-13 07:30:00.000|2017-11-13 07:30:00|
|2017-11-12T19:05:24.000Z|2017-11-12 19:05:24.000|2017-11-12 19:05:24|
|2017-11-13T11:00:04.000Z|2017-11-13 11:00:04.000|2017-11-13 11:00:04|
|2017-11-12T18:01:41.000Z|2017-11-12 18:01:41.000|2017-11-12 18:01:41|
+------------------------+-----------------------+-------------------+
only showing top 5 rows



<div class="alert alert-block alert-warning"><b>Translate function</b></div>

In [291]:
videos.select("publish_time", translate(col("publish_time"),"TZ", " ").alias("trans")).show(5,False)

+------------------------+-----------------------+
|publish_time            |trans                  |
+------------------------+-----------------------+
|2017-11-13T17:13:01.000Z|2017-11-13 17:13:01.000|
|2017-11-13T07:30:00.000Z|2017-11-13 07:30:00.000|
|2017-11-12T19:05:24.000Z|2017-11-12 19:05:24.000|
|2017-11-13T11:00:04.000Z|2017-11-13 11:00:04.000|
|2017-11-12T18:01:41.000Z|2017-11-12 18:01:41.000|
+------------------------+-----------------------+
only showing top 5 rows



Very weird function, it takes one parameter - what to chnage and second is replacement value, but written in one ""..

<div class="alert alert-block alert-warning"><b>Trim function</b></div>

In [292]:
df = df.withColumn("title", trim(df.title))

In [293]:
df.select("title").show(4,False)

+--------------------------------------------------------------+
|title                                                         |
+--------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |
|Nickelback Lyrics: Real or Fake?                              |
+--------------------------------------------------------------+
only showing top 4 rows



<div class="alert alert-block alert-warning"><b>Lower function</b></div>

In [294]:
df.select("title", lower(df.title)).show(5,False)

+--------------------------------------------------------------+--------------------------------------------------------------+
|title                                                         |lower(title)                                                  |
+--------------------------------------------------------------+--------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |we want to talk about our marriage                            |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|the trump presidency: last week tonight with john oliver (hbo)|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |racist superman | rudy mancuso, king bach & lele pons         |
|Nickelback Lyrics: Real or Fake?                              |nickelback lyrics: real or fake?                              |
|I Dare You: GOING BALD!?                                      |i dare you: going bald!?                

<div class="alert alert-block alert-warning"><b>Case When - creating column based on multiple condition</b></div>

In [299]:
#Option 1: when-otherwise
df.select("likes","dislikes", (when(df.likes > df.dislikes, "Good").when\
             (df.likes < df.dislikes, "Bad").otherwise("Undertermined")).alias("Favorability")).show(3)
#we are creating new column which has values good bad and undertermined based on condition

+------+--------+------------+
| likes|dislikes|Favorability|
+------+--------+------------+
| 57527|    2966|        Good|
| 97185|    6146|        Good|
|146033|    5339|        Good|
+------+--------+------------+
only showing top 3 rows



*SQL style*

In [303]:
#Option 2
df.select("likes","dislikes", expr("CASE WHEN likes > dislikes THEN \
'Good' WHEN dislikes > likes THEN 'Bad' ELSE 'Undertermined' END AS Favorability")).show(3)
#It is the same as option 1 but slightly different approach more SQL style

+------+--------+------------+
| likes|dislikes|Favorability|
+------+--------+------------+
| 57527|    2966|        Good|
| 97185|    6146|        Good|
|146033|    5339|        Good|
+------+--------+------------+
only showing top 3 rows



*And again SQL style - filtering based on condition*

In [304]:
df.selectExpr("likes","dislikes", "CASE WHEN likes > dislikes THEN \
'Good' WHEN dislikes > likes THEN 'Bad' ELSE 'Undertermined' END AS Favorability").show(3)

+------+--------+------------+
| likes|dislikes|Favorability|
+------+--------+------------+
| 57527|    2966|        Good|
| 97185|    6146|        Good|
|146033|    5339|        Good|
+------+--------+------------+
only showing top 3 rows



<div class="alert alert-block alert-warning"><b>Concatenate</b></div>

In [306]:
df.select(concat_ws("", df.title, df.channel_title).alias("text")).show(5,False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGECaseyNeistat                               |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)LastWeekTonight|
|Racist Superman | Rudy Mancuso, King Bach & Lele PonsRudy Mancuso            |
|Nickelback Lyrics: Real or Fake?Good Mythical Morning                        |
|I Dare You: GOING BALD!?nigahiga                                             |
+-----------------------------------------------------------------------------+
only showing top 5 rows



*I just concatenate two titles together with no space, nothing else :).*

<div class="alert alert-block alert-warning"><b>Date Extraction</b></div>

In [307]:
df.select("trending_date", year("trending_date"),month("trending_date")).show(5)

+-------------+-------------------+--------------------+
|trending_date|year(trending_date)|month(trending_date)|
+-------------+-------------------+--------------------+
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
|   2017-01-14|               2017|                   1|
+-------------+-------------------+--------------------+
only showing top 5 rows



*Nice, I am easily extracting month and year from our trending date. First selection is from which column we are extracting.*

<div class="alert alert-block alert-warning"><b>Split</b></div>

In [315]:
array = df.select("title", split(df.title, " ").alias("new"))

In [318]:
array.show(5,False)

+--------------------------------------------------------------+-------------------------------------------------------------------------+
|title                                                         |new                                                                      |
+--------------------------------------------------------------+-------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |[WE, WANT, TO, TALK, ABOUT, OUR, MARRIAGE]                               |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|[The, Trump, Presidency:, Last, Week, Tonight, with, John, Oliver, (HBO)]|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |[Racist, Superman, |, Rudy, Mancuso,, King, Bach, &, Lele, Pons]         |
|Nickelback Lyrics: Real or Fake?                              |[Nickelback, Lyrics:, Real, or, Fake?]                                   |
|I Dare You: GOING BALD!?  

*I am splitting all of my words in title with space and automatically with a comma.*

<div class="alert alert-block alert-warning"><b>Array distinct, contains</b></div>

In [319]:
array.select("title", array_contains(array.new, "MARRIAGE")).show(1,False)

+----------------------------------+-----------------------------+
|title                             |array_contains(new, MARRIAGE)|
+----------------------------------+-----------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE|true                         |
+----------------------------------+-----------------------------+
only showing top 1 row



*Siímple function which creates colum with bol values if our column title contains specific expression.*

In [320]:
array.select("title", array_distinct(array.new)).show(5,False)

+--------------------------------------------------------------+-------------------------------------------------------------------------+
|title                                                         |array_distinct(new)                                                      |
+--------------------------------------------------------------+-------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |[WE, WANT, TO, TALK, ABOUT, OUR, MARRIAGE]                               |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|[The, Trump, Presidency:, Last, Week, Tonight, with, John, Oliver, (HBO)]|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |[Racist, Superman, |, Rudy, Mancuso,, King, Bach, &, Lele, Pons]         |
|Nickelback Lyrics: Real or Fake?                              |[Nickelback, Lyrics:, Real, or, Fake?]                                   |
|I Dare You: GOING BALD!?  

*Similar to our split function.*

In [321]:
array.select("title", array_remove(array.new, "WE")).show(5,False)

+--------------------------------------------------------------+-------------------------------------------------------------------------+
|title                                                         |array_remove(new, WE)                                                    |
+--------------------------------------------------------------+-------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                            |[WANT, TO, TALK, ABOUT, OUR, MARRIAGE]                                   |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)|[The, Trump, Presidency:, Last, Week, Tonight, with, John, Oliver, (HBO)]|
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons         |[Racist, Superman, |, Rudy, Mancuso,, King, Bach, &, Lele, Pons]         |
|Nickelback Lyrics: Real or Fake?                              |[Nickelback, Lyrics:, Real, or, Fake?]                                   |
|I Dare You: GOING BALD!?  

*I am just removing word WE from each row.*

<div class="alert alert-block alert-warning"><b>Functions - UDF </b></div>

**Functions in PySpark - different from Python, because they work on Cluster**

In [323]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

*I am just importing udf function and datatype I wanna return.*

In [324]:
def square(x):
    return int(x**2) #easy python function

In [327]:
square_udf = udf(lambda z: square(z), IntegerType())
#I am creting lambda function which is UDF function, which is calling my regular python square function

In [329]:
df.select("dislikes", square_udf("dislikes")).where(col("dislikes").isNotNull()).show(5)
#I have to use where to make this function working, whic is quite weird..so its mandatory to use where...

+--------+------------------+
|dislikes|<lambda>(dislikes)|
+--------+------------------+
|    2966|           8797156|
|    6146|          37773316|
|    5339|          28504921|
|     666|            443556|
|    1989|           3956121|
+--------+------------------+
only showing top 5 rows



<div class="alert alert-block alert-danger"><b>BeAware:</b>I have to register functions as UDF only if they operate on columns, if they operate on whole dataset I can use regular Python functions</div>

### Aggregating Data in Dataframes

- GroupBy
- Pivot
- Aggregate Methods

<div class="alert alert-block alert-warning"><b>Starting Session</b></div>

In [331]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aggregate").getOrCreate()

In [332]:
spark

<div class="alert alert-block alert-warning"><b>Data and basic change</b></div>

In [333]:
airbnb = spark.read.csv("Datasets/nyc_air_bnb.csv", inferSchema = True, header = True)

In [335]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [336]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [337]:
from pyspark.sql.types import *
from pyspark.sql.functions import*

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType()))\
.withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType()))\
.withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType()))\
.withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType()))\
.withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))

In [338]:
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


<div class="alert alert-block alert-warning"><b>GroupBy </b></div>

In [339]:
df.groupBy("neighbourhood_group").count().show(7)

+-------------------+-----+
|neighbourhood_group|count|
+-------------------+-----+
|         Douglaston|    1|
|             Queens| 5630|
|              Nadia|    1|
|            Midtown|    4|
|    Jackson Heights|    2|
|     Hell's Kitchen|    7|
|  Greenwich Village|    2|
+-------------------+-----+
only showing top 7 rows



*I am just grouping based on neighbourhood group and counting values.*

In [340]:
df.groupBy("neighbourhood").agg({"price": "mean"}).show(5)

+-------------+----------+
|neighbourhood|avg(price)|
+-------------+----------+
|       Corona| 59.171875|
| Richmondtown|      78.0|
| Prince's Bay|     409.5|
|  Westerleigh|      71.5|
|   Mill Basin|    179.75|
+-------------+----------+
only showing top 5 rows



*Second style of writing it..nothing else.*

In [343]:
df.groupBy("neighbourhood").agg(min(df.price), max(df.price)).limit(5).toPandas()

,neighbourhood,min(price),max(price)
0,Corona,23,359
1,Richmondtown,78,78
2,Prince's Bay,85,1250
3,Westerleigh,40,103
4,Mill Basin,85,299


*This way I can use multiple grouping, not so easy as in Pandas.*

In [344]:
df.summary("count","min","max","25%", "75%").limit(5).toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,count,49079,49047,48894,48873,48894,48894,48894,48894,48894,48887,48891,48738,38845,38858,48891,48737
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-74,0,0,-73.94134,0,0,0
2,max,"獨一無二的紐約閣樓""","ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",呈刚,현선,Woodside,Woodside,West Village,Shared room,Shared room,10000,1250,629,9.66,58,365,365
3,25%,9471893.0,2.4544724E7,7797690.0,475.0,1.94716858E8,40.68771,40.68981,-73.98309,56.0,69,1,1,0.76,0,1,0
4,75%,2.9152899E7,1.74786681E8,1.07434423E8,NaN,1.97400421E8,40.78304,40.76299,-73.93638,145.0,175,5,23,3.24,2,2,226


*It is very neat function, but its trying to do summary also on non numeric columns :). I have to write my columns seperately for this purpose.*

In [346]:
df.select(countDistinct("neighbourhood_group"),avg("price"),stddev("price")).limit(5).toPandas()

,count(DISTINCT neighbourhood_group),avg(price),stddev_samp(price)
0,77,152.222984,238.541467


*Another grouping function.*

<div class="alert alert-block alert-warning"><b>Pivot</b></div>

In [349]:
df.groupBy("room_type").pivot("neighbourhood_group", ["Queens","Brooklyn"]).count().limit(15).toPandas()

,room_type,Queens,Brooklyn
0,51,NaN,NaN
1,205,NaN,NaN
2,54,NaN,NaN
3,200,NaN,NaN
4,279,NaN,NaN
5,138,NaN,NaN
6,69,NaN,NaN
7,42,NaN,NaN
8,Shared room,198.0,413.0
9,-73.95777,NaN,NaN


*Hmm this is probably bad example for pivoting.*

### Joining and Appending DataFrames

In [350]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Joins").getOrCreate()
spark

<div class="alert alert-block alert-warning"><b>Data</b></div>

*Firstly two simple tables, just for this purpose.*

In [352]:
valuesP = [("koala",1,"yes"), ("caterpillar",2,"yes"),("deer",3,"yes"),("human",4,"yes")]
eats_plants = spark.createDataFrame(valuesP, ["name","id", "eats_plants"])

valuesM = [("shark",5,"yes"), ("lion",6,"yes"), ("tiger",7,"yes"),("human",4,"yes")]
eats_meat = spark.createDataFrame(valuesM, ["name", "id", "eats_meat"])

print("Plant eaters (herbivores)")
print(eats_plants.show())
print("Meat eaters (carnivores)")
print(eats_meat.show())

Plant eaters (herbivores)
+-----------+---+-----------+
|       name| id|eats_plants|
+-----------+---+-----------+
|      koala|  1|        yes|
|caterpillar|  2|        yes|
|       deer|  3|        yes|
|      human|  4|        yes|
+-----------+---+-----------+

None
Meat eaters (carnivores)
+-----+---+---------+
| name| id|eats_meat|
+-----+---+---------+
|shark|  5|      yes|
| lion|  6|      yes|
|tiger|  7|      yes|
|human|  4|      yes|
+-----+---+---------+

None


<div class="alert alert-block alert-warning"><b>Joining</b></div>

***Inner***

In [357]:
inner_join = eats_plants.join(eats_meat, ["name", "id"], "inner")

In [358]:
inner_join.show()

+-----+---+-----------+---------+
| name| id|eats_plants|eats_meat|
+-----+---+-----------+---------+
|human|  4|        yes|      yes|
+-----+---+-----------+---------+



*Its just inner joing so I have only human who is eating vegi and meat.*

***Left***

In [360]:
left_join = eats_plants.join(eats_meat, ["name", "id"], "left")

In [361]:
left_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      human|  4|        yes|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



*It just connect the same ids nothing else.*

***CONDITIONAL JOIN***

In [363]:
cond_join = eats_plants.join(eats_meat, ["name", "id"], "left")\
.filter(eats_meat.name.isNull())

In [365]:
cond_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



Joining + condition in this case null values in eats meat.

***Full outer join***

In [368]:
full_join = eats_plants.join(eats_meat, ["name", "id"], "full")
full_join.show()

+-----------+---+-----------+---------+
|       name| id|eats_plants|eats_meat|
+-----------+---+-----------+---------+
|       deer|  3|        yes|     null|
|      shark|  5|       null|      yes|
|      human|  4|        yes|      yes|
|      tiger|  7|       null|      yes|
|       lion|  6|       null|      yes|
|      koala|  1|        yes|     null|
|caterpillar|  2|        yes|     null|
+-----------+---+-----------+---------+



*Just joining everything together.*

<div class="alert alert-block alert-warning"><b>Real data practice</b></div>

In [369]:
import os
path = "Datasets/uw-madison-courses/"

In [372]:
df_list = []
for filename in os.listdir(path): #i iterate in my folder..
    if filename.endswith(".csv"): #looking for csv
        filename_list = filename.split(".") #for ma name i am splitting csv ending
        df_name = filename_list[0] #and creating a name
        df = spark.read.csv(path+filename, inferSchema=True, header=True) #my spark is reading the file
        df.name = df_name #setting tne name again
        df_list.append(df_name) #and adding to my list
        exec(df_name + " = df") #pff dunno what this is doing

In [376]:
df_list #now i have all my csv in list and can call them

['courses',
 'course_offerings',
 'grade_distributions',
 'instructors',
 'rooms',
 'schedules',
 'sections',
 'subjects',
 'subject_memberships',
 'teachings']

In [382]:
course_offerings.show(3,False) #TABLE 1 

+------------------------------------+------------------------------------+---------+--------------------------+
|uuid                                |course_uuid                         |term_code|name                      |
+------------------------------------+------------------------------------+---------+--------------------------+
|344b3ebe-da7e-314c-83ed-9425269695fd|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1092     |Cooperative Education Prog|
|f718e6cd-33f0-3c14-a9a6-834d9c3610a8|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1082     |Cooperative Education Prog|
|ea3b717c-d66b-30dc-8b37-964d9688295f|a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de|1172     |Cooperative Education Prog|
+------------------------------------+------------------------------------+---------+--------------------------+
only showing top 3 rows



*Pretty handy function.*

In [383]:
instructors.show(3,False) #TABLE 2

+-------+----------------+
|id     |name            |
+-------+----------------+
|761703 |JOHN ARCHAMBAULT|
|3677061|STEPHANIE KANN  |
|788586 |KATHY PREM      |
+-------+----------------+
only showing top 3 rows



In [384]:
sections.show(3,False) #TABLE3

+------------------------------------+------------------------------------+------------+------+------------------------------------+------------------------------------+
|uuid                                |course_offering_uuid                |section_type|number|room_uuid                           |schedule_uuid                       |
+------------------------------------+------------------------------------+------------+------+------------------------------------+------------------------------------+
|45adf63c-48c9-3659-8561-07556d2d4ddf|344b3ebe-da7e-314c-83ed-9425269695fd|FLD         |1     |null                                |f2d66a4d-0c08-3b48-abf6-649fffd7ae90|
|c6280e23-5e43-3859-893e-540d94993529|f718e6cd-33f0-3c14-a9a6-834d9c3610a8|FLD         |1     |null                                |f2d66a4d-0c08-3b48-abf6-649fffd7ae90|
|9395dc21-15d1-3fab-8d1f-6f3fe6114c48|ea3b717c-d66b-30dc-8b37-964d9688295f|FLD         |1     |04368a56-c959-3e4b-8b3d-f4cc3538fea5|f2d66a4d-0c08-3b48

In [385]:
teachings.show(3,False) #TABLE4

+-------------+------------------------------------+
|instructor_id|section_uuid                        |
+-------------+------------------------------------+
|761703       |45adf63c-48c9-3659-8561-07556d2d4ddf|
|761703       |c6280e23-5e43-3859-893e-540d94993529|
|761703       |9395dc21-15d1-3fab-8d1f-6f3fe6114c48|
+-------------+------------------------------------+
only showing top 3 rows



I wanna join instructor with course, I have to use 4 tables

In [387]:
step1 = teachings.join(instructors, teachings.instructor_id == instructors.id, "left")\
.select(["instructor_id", "name","section_uuid"]) #I wanna only these three columns
step1.show(5)
#So my teacher has id and name now

+-------------+----------------+--------------------+
|instructor_id|            name|        section_uuid|
+-------------+----------------+--------------------+
|       761703|JOHN ARCHAMBAULT|45adf63c-48c9-365...|
|       761703|JOHN ARCHAMBAULT|c6280e23-5e43-385...|
|       761703|JOHN ARCHAMBAULT|9395dc21-15d1-3fa...|
|      3677061|  STEPHANIE KANN|b99e440b-39db-350...|
|       761703|JOHN ARCHAMBAULT|ca1c841f-41d5-329...|
+-------------+----------------+--------------------+
only showing top 5 rows



In [389]:
step2 = step1.join(sections, step1.section_uuid == sections.uuid, "left")\
.select(["name", "course_offering_uuid"])
step2.limit(4).toPandas()

,name,course_offering_uuid
0,THOMAS JAHNS,f850ab24-740c-311a-a669-804a3fea7b0b
1,JEAN-FRANCOIS HOUDE,7e213b2b-c58b-3014-b3d1-01c0f7ed46ef
2,CHRISTOPHER R TABER,3beb7bd7-4877-3c63-8afc-62f8b74e72fc
3,MARISA S OTEGUI,db253216-2e66-3267-86b2-7b9f5fe07223


In [392]:
step3 = step2.withColumnRenamed("name", "instructor").join(course_offerings, \
                                                           step2.course_offering_uuid == course_offerings.uuid,"inner")\
.select(["name","instructor", "course_uuid"])

In [393]:
step3.limit(4).toPandas()

,name,instructor,course_uuid
0,Master's Research or Thesis,THOMAS JAHNS,2c3a2a38-9f53-3cfb-8f37-cde4cb3d4b4b
1,Wrkshp-Industrl Organizatn,JEAN-FRANCOIS HOUDE,622a73dc-c070-38e4-8ba8-2b2c3f7a1056
2,Workshop - Public Economics,CHRISTOPHER R TABER,685a727c-fdc0-3ba6-b521-1f236f69b5fa
3,Plant Cell Biology,MARISA S OTEGUI,abc31e90-8a8f-37bf-9fe0-bacd53aa2a1f


***And thats it, more complicated but okay..just joining more tables to achieve one result.***

<div class="alert alert-block alert-warning"><b>Levenshtein</b></div>

In [394]:
from pyspark.sql.functions import levenshtein

In [397]:
df0 = spark.createDataFrame([("Aple","Apple","Microsoft","IBM")], ["Input","Option1","Option2"])

In [398]:
df0.show()

+-----+-------+---------+---+
|Input|Option1|  Option2| _4|
+-----+-------+---------+---+
| Aple|  Apple|Microsoft|IBM|
+-----+-------+---------+---+



In [399]:
df0.select(levenshtein("Input", "Option1").alias("Apple")).show()

+-----+
|Apple|
+-----+
|    1|
+-----+



In [400]:
df0.select(levenshtein("Input", "Option2").alias("Microsoft")).show()

+---------+
|Microsoft|
+---------+
|        9|
+---------+



Its really weird function, counting distance from input for our alias in our option.

### Handling Missing Data

In [401]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Nan").getOrCreate()
spark

<div class="alert alert-block alert-warning"><b>Basic Data</b></div>

In [402]:
zomato = spark.read.csv("Datasets/zomato.csv", inferSchema=True, header=True)

In [415]:
zomato.limit(2).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,None,None,None,None,None,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"""[('Rated 4.0', 'RATED\n A beautiful place to...",('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...


In [405]:
zomato.printSchema()

root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): string (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)



In [409]:
from pyspark.sql.types import*
from pyspark.sql.functions import*
                                       
df = zomato.withColumn("approx_cost(for two people)", zomato["approx_cost(for two people)"].cast(IntegerType()))\
.withColumn("votes", zomato["votes"].cast(IntegerType()))

print(df.printSchema())

root
 |-- url: string (nullable = true)
 |-- address: string (nullable = true)
 |-- name: string (nullable = true)
 |-- online_order: string (nullable = true)
 |-- book_table: string (nullable = true)
 |-- rate: string (nullable = true)
 |-- votes: integer (nullable = true)
 |-- phone: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rest_type: string (nullable = true)
 |-- dish_liked: string (nullable = true)
 |-- cuisines: string (nullable = true)
 |-- approx_cost(for two people): integer (nullable = true)
 |-- reviews_list: string (nullable = true)
 |-- menu_item: string (nullable = true)
 |-- listed_in(type): string (nullable = true)
 |-- listed_in(city): string (nullable = true)

None


In [414]:
df.limit(2).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775.0,080 42297555,None,None,None,None,NaN,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,NaN,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',NaN,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...


*Now a little search for missing data.*

In [413]:
df.filter(df.cuisines.isNull()).select(["name","cuisines"]).show(5)

+---------------+--------+
|           name|cuisines|
+---------------+--------+
|          Jalsa|    null|
|  Grand Village|    null|
|  Casual Dining|    null|
|Timepass Dinner|    null|
|  Casual Dining|    null|
+---------------+--------+
only showing top 5 rows



<div class="alert alert-block alert-warning"><b>Counter for missing data</b></div>

*I guess PySpark doesn't have info function :).*

In [416]:
from pyspark.sql.functions import *

In [435]:
def null_value_calc(df):
    null_columns_counts = [] #number of null values - list
    numRows = df.count() #counting number of rows
    for k in df.columns: #iterate throug columns
        nullRows = df.where(col(k).isNull()).count() #counting null values
        if(nullRows>0): #if I have some
            temp = k,nullRows, ((nullRows/numRows)*100) #I calculcate percentage
            null_columns_counts.append(temp) # and add them to our list
    return(null_columns_counts)

In [436]:
null_columns_calc_list = null_value_calc(df)

Now I have number of missing values in my table and also in percentage.

In [437]:
null_columns_calc_list

[('name', 85, 0.11849993029415865),
 ('online_order', 8111, 11.307681583716715),
 ('book_table', 2, 0.0027882336539802035),
 ('rate', 7775, 10.839258329848041),
 ('votes', 20018, 27.907430642687856),
 ('phone', 1227, 1.7105813467168547),
 ('location', 20054, 27.957618848459504),
 ('rest_type', 20165, 28.1123658162554),
 ('dish_liked', 46841, 65.30182629304335),
 ('cuisines', 27305, 38.06635996096473),
 ('approx_cost(for two people)', 43611, 60.798828941865324),
 ('reviews_list', 28185, 39.293182768716015),
 ('menu_item', 28611, 39.8870765370138),
 ('listed_in(type)', 28983, 40.40568799665412),
 ('listed_in(city)', 29344, 40.908964171197546)]

In [439]:
spark.createDataFrame(null_columns_calc_list, ["Name", "Count", "Percent"]).limit(30).toPandas()

,Name,Count,Percent
0,name,85,0.118500
1,online_order,8111,11.307682
2,book_table,2,0.002788
3,rate,7775,10.839258
4,votes,20018,27.907431
5,phone,1227,1.710581
6,location,20054,27.957619
7,rest_type,20165,28.112366
8,dish_liked,46841,65.301826
9,cuisines,27305,38.066360


*Just for a better look.*

<div class="alert alert-block alert-warning"><b>Dropping Nan</b></div>

In [441]:
df.na.drop().limit(4).toPandas()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"""[('Rated 4.0', 'RATED\n Had been here for di...",rice was well cooked and overall was great\n\n...,('Rated 5.0','RATED\n This place just cool ? with good am...
1,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"""[('Rated 3.0', """"RATED\n Ambience is not tha...",('Rated 3.0',"""""RATED\n \nWent there for a quick bite with ...",pasta churros and lasagne.\n\nNachos were pat...
2,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"""[('Rated 4.0', """"RATED\n Great food and prop...",('Rated 2.0','RATED\n Reached the place at 3pm on Saturda...,('Rated 4.0'
3,https://www.zomato.com/bangalore/cafe-shuffle-...,"941, 3rd FLOOR, 21st Main, 22nd Cross, Banasha...",Cafe Shuffle,Yes,Yes,4.2/5,150,+91 9742166777,Banashankari,Cafe,"Mocktails, Peri Fries, Lasagne, Pizza, Chicken...","Cafe, Italian, Continental",600,"""[('Rated 1.0', """"RATED\n \n\nHorrible. Not ev...","you get it literally half an hour late."""")",('Rated 4.0',"""""RATED\n While this place is more common fo..."


*It is deleteing every row which has any Nan in any column, so carefully with that.*

In [455]:
og_len = df.count()
drop_len = df.na.drop().count()
print(og_len-drop_len)

63124


My function would be deleting 63 124 rows...

In [460]:
drop_len = df.na.drop(thresh=8).count()

In [461]:
print(og_len-drop_len)

1694


Only drop rowss if they have more then thres Nan values in rows - so I can see that with thresh 8 I have only 1694 rows for dropping.

In [462]:
drop_len = df.na.drop(subset=["votes"]).count() 
print(og_len-drop_len)

20018


Now I can dropping only if Nan are in specific column. That is probably all I need?

In [464]:
drop_len = df.na.drop(how="all").count() 
#counter for Nan rows where all columns has nan
print(og_len-drop_len)

0


<div class="alert alert-block alert-warning"><b>Fill Nan</b></div>

In [468]:
df.na.fill("MISSING").limit(2).toPandas() 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775.0,080 42297555,MISSING,MISSING,MISSING,MISSING,NaN,MISSING,MISSING,MISSING,MISSING
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,NaN,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',NaN,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...


<div class="alert alert-block alert-danger"><b>BeAware:</b>If I am choosing string to be filled in Nan, I am only filling String data type rows</div>

In [467]:
df.na.fill(999).limit(2).toPandas() #now I am using number so I fill nan also in IntegerType

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555,None,None,None,None,999,None,None,None,None
1,"+91 9743772233""",Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,999,('Rated 4.0','RATED\n You canÃ\x83Ã\x83Ã\x82Ã\x82Ã\x...,('Rated 5.0','RATED\n Overdelighted by the service and fo...,('Rated 4.0',999,('Rated 4.0','RATED\n The place is nice and comfortable. ...,('Rated 4.0','RATED\n The place is nice and comfortable. ...


In [469]:
#SUBSET ALSO POSSIBLE = df.na.fill(999, subset=["Same as before"]).limit(2).toPandas() 

<div class="alert alert-block alert-info"><b>Tip:</b> Handy function for filling NaN with Mean values</div>

In [473]:
def fill_with_mean(df, include=set()): #so I choose my columns I want to fill in
    stats= df.agg(*(avg(c).alias(c) for c in df.columns if c in include)) #I am creating my mean for my columns
    return df.na.fill(stats.first().asDict()) #and filling in

In [474]:
updated_df = fill_with_mean(df, ["votes"]) #Now I use function for filling

***Don't forget to add this function to Notes.***

## Part II MLlib

***Scikit-Learn for PySpark which provides a lot of classical learning algorithms and also support CV, Vectorizor, Train-Validation Split, Pipelines and basic statistics.***

*Documentation for MLlib: https://spark.apache.org/docs/latest/ml-guide.html*

**MLlib require:**

- dense vector format for features
- all features has to be numerical 
- for classification is Spark expecting a 0 indexed label
- for Naive Bays all features has to be non-negative

That is all for introduction. Next I will proceed with MLlib more deeply.